# Using Comet

For tracking and visualize model metrics in real time.

In [ ]:
%pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 51.5 MB/s eta 0:00:00


In [ ]:
import comet_ml
comet_ml.init(project_name='object-countings')

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


# Setup YOLOv5


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
# !pip install -U -r yolov5/requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 16057, done.
remote: Total 16057 (delta 0), reused 0 (delta 0), pack-reused 16057
Receiving objects: 100% (16057/16057), 14.66 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (11028/11028), done.


In [ ]:
# !pip install utils
!pip install -U -r /content/yolov5/requirements.txt

# Setup Dataset

In [ ]:
!curl -L "https://app.roboflow.com/ds/wlBe7G8TD0?key=Gf3xzM64fu" > countings.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0   1672      0 --:--:-- --:--:-- --:--:--  1671
100  252M  100  252M    0     0  17.4M      0  0:00:14  0:00:14 --:--:-- 21.5M


In [ ]:
import zipfile
with zipfile.ZipFile('countings.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/yolov5/countings')

In [ ]:
%cat /content/yolov5/countings/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['cage_wheel_track_down', 'cage_wheel_track_up']

roboflow:
  workspace: materials-detection
  project: cage-wheel-track-rev
  version: 21
  license: CC BY 4.0
  url: https://universe.roboflow.com/materials-detection/cage-wheel-track-rev/dataset/21

In [ ]:
import yaml
with open("/content/yolov5/countings/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/countings/data.yaml
train: /content/yolov5/countings/train/images
val: /content/yolov5/countings/valid/images
test: /content/yolov5/countings/test/images

nc: 2
names: ['cage_wheel_track_down', 'cage_wheel_track_up']

roboflow:
  workspace: materials-detection
  project: cage-wheel-track-rev
  version: 21
  license: CC BY 4.0
  url: https://universe.roboflow.com/materials-detection/cage-wheel-track-rev/dataset/21

In [ ]:
with open(r'/content/yolov5/countings/data.yaml') as file:
    labels_list = yaml.load(file, Loader=yaml.FullLoader)
    label_names = labels_list['names']

# Tuning Hyperparameter

In [ ]:
%cat /content/yolov5/data/hyps/hyp.scratch-high.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
# Hyperparameters for high-augmentation COCO training from scratch
# python train.py --batch 32 --cfg yolov5m6.yaml --weights '' --data coco.yaml --img 1280 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # fo

In [ ]:
%%writetemplate /content/yolov5/data/hyps/hyp.scratch-high.yaml
# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
# Hyperparameters for high-augmentation COCO training from scratch
# python train.py --batch 32 --cfg yolov5m6.yaml --weights '' --data coco.yaml --img 1280 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
copy_paste: 0.1  # segment copy-paste (probability)

In [ ]:
#%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 1 --epochs 50 --data '/content/yolov5/try-cometML/data.yaml' --weights 'yolov5s.pt' --multi-scale

python3: can't open file '/content/train.py': [Errno 2] No such file or directory


# Training Script

In [ ]:
!chmod +w /content/yolov5/countings/train/labels
#%%time
%cd /content/yolov5/
!env COMET_LOG_PER_CLASS_METRICS=true python train.py \
--img 640 \
--batch 6 \
--epochs 50 \
--data '/content/yolov5/countings/data.yaml' \
--hyp 'data/hyps/hyp.scratch-high.yaml' \
--name 'Adam50epoch' \
--optimizer 'AdamW' \
--weights '/content/drive/MyDrive/best.pt' \
--multi-scale

/content/yolov5
2023-11-14 01:57:37.129957: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 01:57:37.130022: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 01:57:37.130061: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x.pt, cfg=, data=/content/yolov5/countings/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=50, batch_size=6, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=True, single_cls=False, optimizer=

In [ ]:
!chmod +w /content/yolov5/countings/train/labels
#%%time
%cd /content/yolov5/
!python train.py \
--img 640 \
--batch 6 \
--epochs 20 \
--data '/content/yolov5/countings/data.yaml' \
--hyp 'data/hyps/hyp.scratch-high.yaml' \
--name 'AdamW wd 5e-3 + scheduler' \
--optimizer 'AdamW' \
--weights 'yolov5x.pt' \
--multi-scale \
--cos-lr \
--cache

/content/yolov5
2023-11-15 01:50:09.411556: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 01:50:09.411610: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 01:50:09.411649: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x.pt, cfg=, data=/content/yolov5/countings/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=20, batch_size=6, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=True, single_cls=False, optimizer=A

# Validation Script


In [ ]:
!python val.py --weights '/content/yolov5/runs/train/AdamW wd 5e-3 + scheduler/weights/best.pt'   --data '/content/yolov5/countings/data.yaml' --img 640 --half

val: data=/content/yolov5/countings/data.yaml, weights=['/content/yolov5/runs/train/AdamW wd 5e-3 + scheduler/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /content/yolov5/countings/valid/labels.cache... 484 images, 11 backgrounds, 0 corrupt: 100% 484/484 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 16/16 [01:49<00:00,  6.86s/it]
                   all        484      58276      0.988      0.943      0.964      0.787
 cage_wheel_track_down        484      29231      0.988     

# Save Model


In [ ]:
!zip -r /content/adam_5e-3_scheduler.zip /content/yolov5/runs

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/val/ (stored 0%)
  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 12%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 5%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 4%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 4%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 4%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 12%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 4%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 28%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 5%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/train/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.download("/content/adam_lr0.01.zip")

# Detect

In [ ]:
%cd /content/yolov5/
!python detect.py --weights '/content/drive/MyDrive/000. Silfaradila/best.pt' --img 640 --conf 0.5 --source /content/drive/MyDrive/test --hide-labels --line-thickness 2

/content/yolov5
detect: weights=['/content/drive/MyDrive/000. Silfaradila/best.pt'], source=/content/drive/MyDrive/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=True, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-240-g84ec8b5 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
image 1/10 /content/drive/MyDrive/test/1.jpg: 640x480 159 cage_wheel_track_downs, 160 cage_wheel_track_ups, 65.5ms
image 2/10 /content/drive/MyDrive/test/10.jpg: 640x480 158 cage_wheel_track_downs, 160 cage_wheel_track_ups, 58.1ms
image 3/10 /content/drive/MyDrive/test/2.jpg: 640x480 158 